In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

model_path = "h:/ML_Models/_gemma/model/gemma-2b-it"  # local folder

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForCausalLM.from_pretrained(model_path)
print("Model loaded.")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
llm = HuggingFacePipeline(pipeline=pipe)

print("Pipeline loaded.")

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_milvus import Milvus

# Embedding model
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

# Connect to your existing collection
vector_store = Milvus(
    embedding_model,
    collection_name="Dogs_Breeds_milvus_EN_1",
    connection_args={"host": "localhost", "port": "19530"}
    #use the vector_field_name parameter only if you have a non-default
    #vector_field_name = 'embeddings'
)
print("Vector store loaded.")

In [ ]:
query_criteria = "dogs for rescue operations"
results = vector_store.similarity_search(
    query_criteria,
    k=5,
)
print(f"Retrieved {len(results)} documents from Milvus for the query: '{query_criteria}'")


In [ ]:
# Use in LangChain RAG
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS

# Convert retrieved documents to FAISS retriever
retriever = FAISS.from_documents(results, embedding=embedding_model).as_retriever()

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff", )

question = "Given the following documents, which docs are good for rescue operations?"

response = qa_chain.run(question)

print(response)
print("Response generated.")

In [ ]:
#Answers using prompt
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are a rescue operations expert. Based on the following documents, identify which ones are most relevant for rescue missions.

    Documents:
    {context}

    Question:
    {question}

    Answer in a clear, concise manner. If none are relevant, say so explicitly.
    """
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # or "map_reduce", "refine" if you want more control
    chain_type_kwargs={"prompt": custom_prompt}
)

# Run the chain
question = "Given the following documents, which docs are good for rescue operations?"
response = qa_chain.run(question)
print(response)
print("Response with prompt generated.")